In [2]:
import ipywidgets as widgets
from IPython.display import display
import requests

class Login:
    def __init__(self, user, host):
        self.user = user
        self.host = host

        employees = requests.get(f"{host}/deceptecons/employee").json()
        employees.sort(key=lambda x: x['eid'])
        employee_options = [f"{employee['eid']}" for employee in employees]

        logins = requests.get(f"{host}/deceptecons/login").json()
        logins.sort(key=lambda x: x['lid'])
        logins = list(map(lambda x: f"{x['lid']}: {x['user']}", filter(lambda x: x['lid'] != -1 and x['eid'] != -1, logins)))

        self.create_login_eid_dropdown = widgets.Dropdown(
            options=employee_options,
            description='EID:',
            ensure_option=True,
            disabled=False
        )
        self.create_login_user_text = widgets.Text(
            value='',
            placeholder='Enter your username',
            description='Username:',
            disabled=False
        )
        self.create_login_password_text = widgets.Password(
            value='',
            placeholder='Enter your password',
            description='Password:',
            disabled=False
        )
        self.submit_create_login = widgets.Button(
            description="Create Login",
            disabled=False,
            button_style='success',
            tooltip="Create Login",
            icon='check'
        )

        self.submit_delete_login = widgets.Button(
            description="Delete Login",
            disabled=False,
            button_style='danger',
            tooltip="Delete Login",
            icon='trash'
        )
        self.submit_update_login = widgets.Button(
            description="Update Login",
            disabled=False,
            button_style='info',
            tooltip="Update Login",
            icon='pencil'
        )
        self.update_login_lid_combobox = widgets.Dropdown(
            placeholder='Choose One',
            options=logins,
            description='Update Login:',
            ensure_option=True,
            disabled=False
        )
        self.update_login_eid_dropdown = widgets.Dropdown(
            options=employee_options,
            description='EID:',
            ensure_option=True,
            disabled=False
        )
        self.update_login_user_text = widgets.Text(
            value='',
            placeholder='Enter your username',
            description='Username:',
            disabled=False
        )
        self.update_login_password_text = widgets.Password(
            value='',
            placeholder='Enter your password',
            description='Password:',
            disabled=False
        )
        self.delete_login_lid_combobox = widgets.Dropdown(
            placeholder='Choose One',
            options=logins,
            description='Delete Login:',
            ensure_option=True,
            disabled=False
        )

        self.status_label = widgets.Label()

        self.create_login_container = widgets.VBox([
            self.create_login_eid_dropdown,
            self.create_login_user_text,
            self.create_login_password_text,
            self.submit_create_login,
            self.status_label
        ])

        self.delete_login_container = widgets.VBox([
            self.delete_login_lid_combobox,
            self.submit_delete_login,
            self.status_label
        ])

        self.update_login_container = widgets.VBox([
            self.update_login_lid_combobox,
            self.update_login_eid_dropdown,
            self.update_login_user_text,
            self.update_login_password_text,
            self.submit_update_login,
            self.status_label
        ])

        self.submit_create_login.on_click(self.on_submit_create_login)
        self.submit_delete_login.on_click(self.on_submit_delete_login)
        self.submit_update_login.on_click(self.on_submit_update_login)

    def on_submit_create_login(self, sender):
        eid = self.create_login_eid_dropdown.value
        user = self.create_login_user_text.value
        password = self.create_login_password_text.value
        data = {"eid": eid, "username": user, "password": password}
        response = requests.post(f"{self.host}/deceptecons/login", json=data)
        if response.status_code == 200:
            self.status_label.value = "Login created successfully."
        else:
            self.status_label.value = "Failed to create login."

    def on_submit_update_login(self, sender):
        lid = self.update_login_lid_combobox.value.split(":")[0]
        eid = self.update_login_eid_dropdown.value
        user = self.update_login_user_text.value
        password = self.update_login_password_text.value
        data = {"eid": eid, "username": user, "password": password}
        response = requests.put(f"{self.host}/deceptecons/login/{lid}", json=data)
        if response.status_code == 200:
            self.status_label.value = "Login updated successfully."
        else:
            self.status_label.value = "Failed to update login."

    def on_submit_delete_login(self, sender):
        lid = self.delete_login_lid_combobox.value.split(":")[0]
        response = requests.delete(f"{self.host}/deceptecons/login/{lid}")
        if response.status_code == 200:
            self.status_label.value = "Login deleted successfully."
        else:
            self.status_label.value = "Failed to delete login."

def run(user, host, output, purpose):
    x = Login(user, host)
    with output:
        if purpose == "Create":
            display(x.create_login_container)
        elif purpose == "Update":
            display(x.update_login_container)
        elif purpose == "Delete":
            display(x.delete_login_container)